#### Currency Coversion

* This problem is derived from LC 399 evaluate division.

You are given the following parameters:
Rates: ['USD', 'JPY', 110] ['USD', 'AUD', 1.45] ['JPY', 'GBP', 0.0070]
To/From currency: ['GBP', 'AUD']
Output: 1.89

#### BBG - VIMP

In [ ]:
from typing import List, Tuple, Dict
from decimal import Decimal, getcontext
from collections import defaultdict, deque

# Set sufficient precision for chained FX conversions
getcontext().prec = 28


class CurrencyConverter:
    """
    Currency conversion engine using graph traversal.

    - Currencies are graph nodes
    - Conversion rates are weighted edges
    - Supports transitive conversion
    - Uses Decimal for financial precision
    """

    def __init__(self, rates: List[Tuple[str, str, float]]):
        """
        rates: List of (from_currency, to_currency, conversion_rate)
        """
        self.graph: Dict[str, List[Tuple[str, Decimal]]] = defaultdict(list)
        self._build_graph(rates)

    def _build_graph(self, rates: List[Tuple[str, str, float]]) -> None:
        """
        Build a bidirectional weighted graph with reciprocal rates.
        """
        for from_ccy, to_ccy, rate in rates:
            decimal_rate = Decimal(str(rate))
            self.graph[from_ccy].append((to_ccy, decimal_rate))
            self.graph[to_ccy].append((from_ccy, Decimal("1") / decimal_rate))

    def get_rate(self, from_ccy: str, to_ccy: str) -> float:
        """
        Returns conversion rate from from_ccy to to_ccy.
        Returns -1.0 if conversion path does not exist.
        """

        if from_ccy == to_ccy:
            return 1.0

        if from_ccy not in self.graph or to_ccy not in self.graph:
            return -1.0

        queue = deque([(from_ccy, Decimal("1"))])
        visited = {from_ccy}

        while queue:
            current_ccy, current_rate = queue.popleft()

            if current_ccy == to_ccy:
                return float(current_rate)

            for neighbor, weight in self.graph[current_ccy]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, current_rate * weight))

        return -1.0

[1.883116883116883]

In [ ]:
# With Cache

from typing import List, Tuple, Dict
from decimal import Decimal, getcontext
from collections import defaultdict, deque

getcontext().prec = 28


class CurrencyConverter:
    """
    Currency conversion engine using graph traversal + query caching.
    """

    def __init__(self, rates: List[Tuple[str, str, float]]):
        self.graph: Dict[str, List[Tuple[str, Decimal]]] = defaultdict(list)

        # NEW: cache for computed conversions
        self.cache: Dict[Tuple[str, str], Decimal] = {}

        self._build_graph(rates)

    def _build_graph(self, rates: List[Tuple[str, str, float]]) -> None:
        for from_ccy, to_ccy, rate in rates:
            decimal_rate = Decimal(str(rate))

            self.graph[from_ccy].append((to_ccy, decimal_rate))
            self.graph[to_ccy].append((from_ccy, Decimal("1") / decimal_rate))

            # Optional: cache direct rates
            self.cache[(from_ccy, to_ccy)] = decimal_rate
            self.cache[(to_ccy, from_ccy)] = Decimal("1") / decimal_rate

    def get_rate(self, from_ccy: str, to_ccy: str) -> float:
        if from_ccy == to_ccy:
            return 1.0

        if from_ccy not in self.graph or to_ccy not in self.graph:
            return -1.0

        # NEW: cache lookup
        if (from_ccy, to_ccy) in self.cache:
            return float(self.cache[(from_ccy, to_ccy)])

        queue = deque([(from_ccy, Decimal("1"))])
        visited = {from_ccy}

        while queue:
            current_ccy, current_rate = queue.popleft()

            for neighbor, weight in self.graph[current_ccy]:
                if neighbor in visited:
                    continue

                new_rate = current_rate * weight

                # Cache intermediate results
                self.cache[(from_ccy, neighbor)] = new_rate
                self.cache[(neighbor, from_ccy)] = Decimal("1") / new_rate

                if neighbor == to_ccy:
                    self.cache[(from_ccy, to_ccy)] = new_rate
                    self.cache[(to_ccy, from_ccy)] = Decimal("1") / new_rate
                    return float(new_rate)

                visited.add(neighbor)
                queue.append((neighbor, new_rate))

        return -1.0
